# EDA: Amazon Sale Report

Este notebook realiza la limpieza, análisis descriptivo, visualización y conclusiones sobre el dataset **Amazon Sale Report.csv**.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configuración general
pd.set_option('display.max_columns', 200)

# Carga de datos
df = pd.read_csv('Amazon Sale Report.csv', low_memory=False)
print('Shape:', df.shape)
df.head()

## Exploración inicial

In [ ]:
# Información de columnas y nulos
display(df.info())
df.isna().sum().sort_values(ascending=False).head(20)

## Limpieza y preprocesamiento

In [ ]:
# Copia de trabajo
df_clean = df.copy()

# Estandarizar nombres de columnas (strip + reemplazar dobles espacios)
df_clean.columns = (
    df_clean.columns.str.strip()
                    .str.replace('\s+', ' ', regex=True)
)

# Eliminar columnas irrelevantes si existen
cols_drop = [c for c in ['Unnamed: 22', 'promotion-ids'] if c in df_clean.columns]
if cols_drop:
    df_clean = df_clean.drop(columns=cols_drop)

# Parseo de fecha
# El formato del CSV se ve como MM-DD-YY (ej: 04-30-22).
if 'Date' in df_clean.columns:
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%m-%d-%y', errors='coerce')

# Coerción de tipos numéricos con seguridad (usa dtypes anulables de pandas)
for col in ['Qty', 'Amount', 'ship-postal-code']:
    if col in df_clean.columns:
        # Intentar convertir a número
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        # Para códigos postales y Qty preferimos enteros anulables si aplica
        if col in ['Qty', 'ship-postal-code']:
            df_clean[col] = df_clean[col].astype('Int64')

# Trim de strings en columnas categóricas típicas
for col in ['Status', 'Fulfilment', 'Sales Channel ', 'ship-service-level', 'Style', 'SKU',
            'Category', 'Size', 'ASIN', 'Courier Status', 'currency', 'ship-city',
            'ship-state', 'ship-country', 'fulfilled-by']:
    if col in df_clean.columns and df_clean[col].dtype == 'object':
        df_clean[col] = df_clean[col].astype(str).str.strip()

# Normalización de Ship Country, Ship State y Ship City
import re
import numpy as np

def clean_location(s):
    if pd.isna(s):
        return s
    s = str(s).strip(" ,.;:-_/\\'\"()[]{}")
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r",\s*", ", ", s)
    s = s.lstrip(", ").strip()
    if re.fullmatch(r"\d+(\.\d+)?", s) or len(s) <= 1:
        return np.nan
    return s.title()

for col in ['ship-city', 'ship-state', 'ship-country']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(clean_location)

# Marca de nulos en Amount
if 'Amount' in df_clean.columns:
    print("Nulos en Amount:", df_clean['Amount'].isna().sum())
    df_clean['Amount_is_null'] = df_clean['Amount'].isna()

# Duplicados (mantenemos la primera ocurrencia)
df_clean = df_clean.drop_duplicates()

print('Shape después de limpieza:', df_clean.shape)
df_clean.head()

In [ ]:
# Normalizar y asegurar la columna Date para Tableau
# Parsear con formato explícito y contar fallas
if 'Date' in df_clean.columns:
    dt = pd.to_datetime(df_clean['Date'], format='%m-%d-%y', errors='coerce')
    print("Filas con Date inválida (NaT) tras el parseo:", int(dt.isna().sum()))
    print("Rango de fechas (min, max):", dt.min(), dt.max())
    df_clean['Date'] = dt

    # Crear columnas para Tableau
    df_clean['Year_num']   = df_clean['Date'].dt.year
    df_clean['Month_str']  = df_clean['Date'].dt.to_period('M').astype(str)  # '2022-04'
    df_clean['Day_num']    = df_clean['Date'].dt.day
    df_clean['Date_str']   = df_clean['Date'].dt.strftime('%Y-%m-%d')

    # Export-friendly: formatear Date como string ISO (yyyy-mm-dd)
    df_clean['Date_str'] = df_clean['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
# Preparar export para Tableau
import csv

# Asegurar nombres únicos
rename_map = {
    'year': 'Year_num',
    'Year': 'Year_num',
    'month': 'Month_str',
    'Month': 'Month_str',
    'day': 'Day_num',
    'Day': 'Day_num',
}
df_clean = df_clean.rename(columns={k:v for k,v in rename_map.items() if k in df_clean.columns})

# Chequeo de unicidad
n_cols = len(df_clean.columns)
n_unique = df_clean.columns.nunique()
print(f"Columnas totales: {n_cols} | únicas: {n_unique}")
assert n_cols == n_unique, "Hay nombres de columna repetidos. Revisa rename_map o renombra manualmente."

# Ordenar columnas (opcional)
cols_order = list(df_clean.columns)  
df_out = df_clean[cols_order].copy()

# Asegurar formatos “Tableau-friendly”
# - Fecha ISO como string
if 'Date' in df_out.columns:
    df_out['Date_str'] = df_out['Date'].dt.strftime('%Y-%m-%d')

# Exportar en **TSV** (TAB) para evitar conflictos de coma/punto y coma
tsv_path = "Amazon_Sale_Report_CLEAN_TABLEAU.tsv"
df_out.to_csv(
    tsv_path,
    sep='\t',               # DELIMITADOR TAB
    index=False,
    encoding='utf-8',
    quoting=csv.QUOTE_MINIMAL,
    line_terminator='\n'
)
print("Exportado:", tsv_path)

# Export 'sanity' con SOLO cabecera para testear columnas en Tableau
header_only = "HEADER_ONLY_TABLEAU.tsv"
pd.DataFrame(columns=df_out.columns).to_csv(
    header_only, sep='\t', index=False, encoding='utf-8'
)
print("Cabecera exportada:", header_only)

In [ ]:
print('Info post-limpieza:')
display(df_clean.info())
print('\nNulos post-limpieza (top 20):')
df_clean.isna().sum().sort_values(ascending=False).head(20)

## Estadísticas descriptivas

In [ ]:
# Numéricas
desc_num = df_clean.select_dtypes(include=['number']).describe().T
display(desc_num)

# Frecuencias categóricas clave
for col in ['Status', 'Category', 'Fulfilment', 'Sales Channel ', 'ship-city', 'ship-state']:
    if col in df_clean.columns:
        print(f'\nTop valores en {col}:')
        display(df_clean[col].value_counts(dropna=False).head(15))

## Análisis temporal

In [ ]:
if 'Date' in df_clean.columns:
    df_clean['year'] = df_clean['Date'].dt.year
    df_clean['month'] = df_clean['Date'].dt.to_period('M').astype(str)
    df_clean['day'] = df_clean['Date'].dt.date

    # Ingresos diarios
    if {'Date','Amount'}.issubset(df_clean.columns):
        daily_rev = df_clean.groupby('Date', dropna=True)['Amount'].sum().sort_index()
        display(daily_rev.head())

        # Gráfico - usar matplotlib y no fijar colores específicos
        plt.figure(figsize=(12,5))
        plt.plot(daily_rev.index, daily_rev.values)
        plt.title('Ingresos diarios')
        plt.xlabel('Fecha'); plt.ylabel('Monto')
        plt.tight_layout(); plt.show()

    # Ingresos por mes
    if {'month','Amount'}.issubset(df_clean.columns):
        monthly_rev = df_clean.groupby('month')['Amount'].sum().sort_index()
        display(monthly_rev.tail())

        plt.figure(figsize=(10,4))
        plt.plot(monthly_rev.index, monthly_rev.values)
        plt.title('Ingresos mensuales')
        plt.xticks(rotation=45)
        plt.xlabel('Mes'); plt.ylabel('Monto')
        plt.tight_layout(); plt.show()

## Patrones por categoría, estado y fulfilment

In [ ]:
def bar_top(series, title, top=10):
    vc = series.value_counts().head(top)
    plt.figure(figsize=(8,4))
    plt.bar(vc.index.astype(str), vc.values)
    plt.title(title)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

if 'Status' in df_clean.columns:
    bar_top(df_clean['Status'], 'Pedidos por estado')
if 'Category' in df_clean.columns:
    bar_top(df_clean['Category'], 'Pedidos por categoría')
if 'Fulfilment' in df_clean.columns:
    bar_top(df_clean['Fulfilment'], 'Pedidos por tipo de fulfilment')
if 'ship-city' in df_clean.columns:
    bar_top(df_clean['ship-city'], 'Top 10 ciudades con más pedidos', top=10)

## Métricas clave de negocio

In [ ]:
metrics = {}
if 'Amount' in df_clean.columns:
    metrics['Ingresos totales'] = float(np.nansum(df_clean['Amount']))
if {'Qty','Amount'}.issubset(df_clean.columns):
    metrics['Ticket promedio (Amount por fila válida)'] = float(np.nanmean(df_clean.loc[df_clean['Amount'].notna(), 'Amount']))
if 'Status' in df_clean.columns:
    total_orders = len(df_clean)
    cancelled = int((df_clean['Status'] == 'Cancelled').sum())
    metrics['% Cancelados'] = round(100*cancelled/total_orders, 2) if total_orders else np.nan

metrics

## Insights y observaciones

- **Calidad de datos**: Se detectaron valores nulos en columnas clave (por ejemplo, códigos postales). Se usaron dtypes anulables (`Int64`) para conservar filas sin forzar imputaciones.
- **Temporalidad**: La columna `Date` fue parseada con formato `MM-DD-YY` y `errors='coerce'` para robustez.
- **Ingresos**: Revisar picos y caídas en el gráfico de ingresos diarios/mensuales para asociarlos a campañas o eventos.
- **Cancelaciones**: Ver el porcentaje de pedidos `Cancelled` y cruzarlo con `Fulfilment` o `ship-city` para encontrar focos de mejora logística.
- **Top categorías/ciudades**: Observar qué categorías y ciudades concentran más pedidos para priorizar inventario y campañas.
- **Próximos pasos**: Segmentar por `Courier Status`, analizar `Qty x Amount`, y detectar outliers en montos/qty; agregar mapa si se dispone de coordenadas.

## Export del dataset limpio

In [ ]:
# Exportar el dataset limpio para importar en Tableau
df_clean['Amount'] = df_clean['Amount'].round(2) # asegurar formato correcto
df_clean.to_csv("Amazon_Sale_Report_CLEAN.csv", index=False, sep=';', decimal=',') # Usar ; como separador y , como decimal
'Archivo exportado: Amazon_Sale_Report_CLEAN.csv'